In [146]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import talib as ta
from datetime import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()
from sklearn.metrics import accuracy_score, r2_score
from datetime import datetime as dt, timedelta

In [147]:
def get_prices(ticker, start_date, end_date):
    start = datetime.strptime(start_date, '%d/%m/%Y')
    end = datetime.strptime(end_date, '%d/%m/%Y')
    f = web.DataReader(ticker, 'morningstar', start, end)
    df = pd.DataFrame(f)
    
    return df

## SQL Connection

In [194]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime

def create_connection():
    # Login details: 
    DSN_Name = 'traderDSN' ; Login_ID = 'basic' ; pwd = 'pwd1'

    # The engine does the same job as a connection and a cursor
    engine = create_engine(r'mssql+pyodbc://'+Login_ID+':'+pwd+'@'+ DSN_Name) 

    return engine 

# Build query here:
query = ''' 
SELECT * FROM {schema}.{table_name}

'''.format(schema='stocks', table_name='Adjusted_NYSE_2')
print('Query:', query)
print('='*60)


engine = create_connection()

# Importing a Pandas Dataframe: 
df1 = pd.read_sql(query,engine)

df1.head()

Query:  
SELECT * FROM stocks.Adjusted_NYSE_2




,index,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,date,Name,Ticker
0,0,188.0000,189.2700,187.1000,187.5800,187.5800,9844406,0.0000,1.0000,2018-05-21,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL
1,0,187.1900,187.8102,186.1300,186.3100,186.3100,18297728,0.0000,1.0000,2018-05-18,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL
2,0,188.0000,188.9100,186.3600,186.9900,186.9900,17294029,0.0000,1.0000,2018-05-17,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL
3,0,186.0700,188.4600,186.0000,188.1800,188.1800,19183064,0.0000,1.0000,2018-05-16,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL
4,0,186.7800,187.0700,185.1000,186.4400,186.4400,23695159,0.0000,1.0000,2018-05-15,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL


In [195]:
prices = df1.copy()

prices = prices.drop(columns=['index','close', 'split coefficient','dividend amount'])
prices = prices.rename(columns={'Ticker':'symbol','adjusted close':'close','Name':'name'})
prices.head()

,open,high,low,close,volume,date,name,symbol
0,188.0000,189.2700,187.1000,187.5800,9844406,2018-05-21,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL
1,187.1900,187.8102,186.1300,186.3100,18297728,2018-05-18,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL
2,188.0000,188.9100,186.3600,186.9900,17294029,2018-05-17,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL
3,186.0700,188.4600,186.0000,188.1800,19183064,2018-05-16,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL
4,186.7800,187.0700,185.1000,186.4400,23695159,2018-05-15,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL


In [196]:
df1.shape

(696115, 12)

In [197]:
    %%time 
    
    # Create engine and query 
    engine = create_connection()
    query = ''' 
    SELECT * FROM {schema}.{table_name}
    '''.format(schema='stocks', table_name='Adjusted_NYSE_2')

    # Importing a Pandas Dataframe
    prices = pd.read_sql(query,engine)
    prices = prices.drop(columns=['index','close', 'split coefficient','dividend amount'])
    prices = prices.rename(columns={'Ticker':'symbol','adjusted close':'close','Name':'name'})
    
    # Reformat column types
    prices['open']  = prices['open'].astype(float)
    prices['high']  = prices['high'].astype(float)
    prices['low']   = prices['low'].astype(float)
    prices['close'] = prices['close'].astype(float)
    prices['volume']= prices['volume'].astype(float)

    prices['date'] = pd.to_datetime(prices['date'])

    prices.set_index(['symbol', 'name', 'date'], inplace=True)
    prices = prices.sort_index(level=1, ascending=True)

Wall time: 8.61 s


In [198]:
prices.head()

open  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  25.50   
                                                          2000-01-04  25.00   
                                                          2000-01-05  24.75   
                                                          2000-01-06  24.00   
                                                          2000-01-07  23.88   

                                                                       high  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  25.50   
                                                          2000-01-04  25.00   
                                                          2000-01-05  24.81   
                                                          2000-01-06  24.00   
                                                          2000-01-07  23.88   

                                                                        low  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  24.75   
                                                          2000-01-04  24.75   
                                                          2000-01-05  23.75   
                                                          2000-01-06  23.63   
                                                          2000-01-07  23.50   

                                                                        close  \
symbol name                                               date                  
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  14.0034   
                                                          2000-01-04  13.8634   
                                                          2000-01-05  13.4433   
                                                          2000-01-06  13.3032   
                                                          2000-01-07  13.2360   

                                                                       volume  
symbol name                                               date                 
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  17800.0  
                                                          2000-01-04   6300.0  
                                                          2000-01-05  20400.0  
                                                          2000-01-06   4200.0  
                                                          2000-01-07   9200.0

In [199]:
    # Calculated columns
    prices  = prices.copy().loc[(prices.loc[:,'open':'volume'] != 0 ).all(axis=1)] # Don't touch this (:
    
    # Group over the symbols. 
    prices_g = prices.groupby(level=['symbol'])
    
    EMA_lengths = [20, 50, 200]  # Calculate many moving averages, of closing prices
    for l in EMA_lengths: 
        prices['EMA_'+str(l)] = prices_g.close.apply(lambda x: ta.EMA(x,l))   

    prices['EMA_ratio'] = prices['EMA_200']/prices['EMA_50'] # Ratio of moving average
    prices['RSI'] = prices_g.close.apply(lambda x: ta.RSI(x, 14))  # 14 day relative strength index

    def CCI_func(x):
        xh = x.high 
        xl = x.low 
        xc = x.close 
        return ta.CCI(high=xh, low=xl, close=xc, timeperiod=5)
   
    prices['CCI'] = prices_g.apply(CCI_func).reset_index(level=1,).drop(columns='symbol')    # created an extra column. 

    prices['Momentum'] = prices_g.close.apply(lambda x: ta.MOM(x, 10))  # 10 day rolling momentum changed to prices_g.close

    def Stoch_fast(x, index):
        xh = x.high 
        xl = x.low 
        xc = x.close 
        return ta.STOCHF(high=xh, low=xl, close=xc, fastk_period=14, fastd_period=3)[index] # Params 14, 3    
    
    prices['Stoch_fastk'] = prices_g.apply(Stoch_fast, 0).reset_index(level=1,).drop(columns='symbol')   # Stochastic fast indicator
    prices['Stoch_fastd'] = prices_g.apply(Stoch_fast, 1).reset_index(level=1,).drop(columns='symbol')

    def OBV_func(x):
        xc = x.close
        xv = x.volume
        return ta.OBV(xc, xv) 
    prices['OBV'] = prices_g.apply(OBV_func).reset_index(level=1,).drop(columns='symbol')  # On Balance Volume
    

    def bband_func(x, index):
        xc = x.close
        return ta.BBANDS(xc)[index]
    
    prices['bbandupper'] = prices_g.apply(bband_func, 0).reset_index(level=1,).drop(columns='symbol')
    prices['bbandmiddle'] = prices_g.apply(bband_func, 1).reset_index(level=1,).drop(columns='symbol') 
    prices['bbandlower'] =  prices_g.apply(bband_func, 2).reset_index(level=1,).drop(columns='symbol')


In [200]:
prices['bbandlowerii'] =  prices['bbandlower']/prices['close']
prices['bbandupperii'] =  prices['bbandlower']/prices['close']

In [240]:
prices.tail(205)

open  \
symbol name                                               date                
WFC    Wells Fargo & Company (WFC) Stock Prices, Divid... 2017-07-28  54.30   
                                                          2017-07-31  53.42   
                                                          2017-08-01  54.23   
                                                          2017-08-02  53.54   
                                                          2017-08-03  53.50   
                                                          2017-08-04  53.67   
                                                          2017-08-07  52.73   
                                                          2017-08-08  52.32   
                                                          2017-08-09  52.28   
                                                          2017-08-10  52.39   
                                                          2017-08-11  51.83   
                                                          2017-08-14  52.37   
                                                          2017-08-15  53.20   
                                                          2017-08-16  53.01   
                                                          2017-08-17  52.47   
                                                          2017-08-18  51.63   
                                                          2017-08-21  51.74   
                                                          2017-08-22  51.95   
                                                          2017-08-23  51.83   
                                                          2017-08-24  52.19   
                                                          2017-08-25  52.27   
                                                          2017-08-28  51.95   
                                                          2017-08-29  51.21   
                                                          2017-08-30  51.46   
                                                          2017-08-31  51.00   
                                                          2017-09-01  51.06   
                                                          2017-09-05  50.63   
                                                          2017-09-06  50.25   
                                                          2017-09-07  49.94   
                                                          2017-09-08  49.61   
...                                                                     ...   
                                                          2018-04-10  52.99   
                                                          2018-04-11  52.32   
                                                          2018-04-12  52.16   
                                                          2018-04-13  52.01   
                                                          2018-04-16  50.59   
                                                          2018-04-17  50.95   
                                                          2018-04-18  50.78   
                                                          2018-04-19  50.51   
                                                          2018-04-20  51.93   
                                                          2018-04-23  52.55   
                                                          2018-04-24  52.95   
                                                          2018-04-25  52.44   
                                                          2018-04-26  52.99   
                                                          2018-04-27  52.28   
                                                          2018-04-30  52.56   
                                                          2018-05-01  51.76   
                                                          2018-05-02  52.38   
                                                          2018-05-03  51.62   
                                          

In [155]:
#US_yields=pd.read_pickle(r'C:\Users\Student\Documents\Trading\US_yieldcurve.pkl')

In [202]:
import quandl
USyields=quandl.get("USTREASURY/YIELD", authtoken="wdLjKc5NXiuwUhFLByKm")
USyields=USyields[['3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '30 YR']]

In [203]:
US_yields.index.rename('date', inplace=True)

In [204]:
prices.reset_index(level='date',inplace=True)

In [205]:
test=pd.merge(prices, US_yields, how='left', left_on='date' , right_index=True)

In [206]:
test.set_index('date',append=True,inplace=True)

In [207]:
test.sort_index(level=1, ascending=True,inplace=True)

In [208]:
test.head()

open  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  25.50   
                                                          2000-01-04  25.00   
                                                          2000-01-05  24.75   
                                                          2000-01-06  24.00   
                                                          2000-01-07  23.88   

                                                                       high  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  25.50   
                                                          2000-01-04  25.00   
                                                          2000-01-05  24.81   
                                                          2000-01-06  24.00   
                                                          2000-01-07  23.88   

                                                                        low  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  24.75   
                                                          2000-01-04  24.75   
                                                          2000-01-05  23.75   
                                                          2000-01-06  23.63   
                                                          2000-01-07  23.50   

                                                                        close  \
symbol name                                               date                  
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  14.0034   
                                                          2000-01-04  13.8634   
                                                          2000-01-05  13.4433   
                                                          2000-01-06  13.3032   
                                                          2000-01-07  13.2360   

                                                                       volume  \
symbol name                                               date                  
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  17800.0   
                                                          2000-01-04   6300.0   
                                                          2000-01-05  20400.0   
                                                          2000-01-06   4200.0   
                                                          2000-01-07   9200.0   

                                                                      EMA_20  \
symbol name                                               date                 
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03     NaN   
                                                          2000-01-04     NaN   
                                                          2000-01-05     NaN   
                                                          2000-01-06     NaN   
                                                          2000-01-07     NaN   

                                                                      EMA_50  \
symbol name                                               date                 
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03     NaN   
                                                          2000-01-04     NaN   
                                                          2000-01-05     NaN   
                                                          2000-01-06     NaN   
                                                          2000-01-07     NaN   

                                                                      EMA_200  \
symbol name                                               date        

In [210]:
### Calculated columns
#prices['EMA_20'] = prices.groupby(level=1).apply(lambda x: ta.EMA(x.close.values, 20))  # 20 day exponential moving average
prices  = test.copy().loc[(test.loc[:,'open':'30 YR'] != 0 ).all(axis=1)] # Don't touch this (:
#prices  = prices.copy().loc[(prices.loc[:,volume'] != 0 ).all(axis=1)] # Don't touch this (:

##price_close = prices.groupby(level=['symbol']).close

##prices['EMA_20'] = price_close.apply(lambda x: ta.EMA(x,20))   # 20 day exponential moving average
##prices['EMA_50'] = price_close.apply(lambda x: ta.EMA(x, 50))  # 50 day exponential moving average
##prices['EMA_200'] = price_close.apply(lambda x: ta.EMA(x, 200))  # 200 day exponential moving average
##prices['EMA_ratio'] = prices['EMA_200']/prices['EMA_50'] # Ratio of moving average
##prices['RSI'] = price_close.apply(lambda x: ta.RSI(x, 14))  # 14 day relative strength index

#prices['CCI'] = prices.apply(lambda x: ta.CCI(high=x.high.va, low=x.low.values, close=x.close.values, timeperiod=20))  # 20 day commodity channel index

##prices['Momentum'] = price_close.apply(lambda x: ta.MOM(x, 10))  # 10 day rolling momentum

#prices['Stoch_fastk'], prices['Stoch_fastd'] = prices.groupby(level=1).apply(lambda x: ta.STOCHF(high=x.high.values, low=x.low.values, close=x.close.values, fastk_period=14, fastd_period=3))  # Stochastic fast indicator
#prices['OBV'] = price_close.apply(lambda x: ta.OBV(np.asarray(x['close']), x['volume']))  # On Balance Volume
#prices['bbandupper'], prices['bbandmiddle'], prices['bbandlower'] =  prices.groupby(level=1).apply(lambda x: ta.BBANDS(x.close.values))


#prices['perc_change']=(prices['close']- prices['close'].shift(1))/prices['close'].shift(1)  # Getting the daily percentage change
#prices['5dayvol'] = prices['perc_change'].rolling(window=5, center=False).std()  # 5 day volatility 
#prices['5_day_target'] = prices.groupby(level=['symbol']).close.shift(-5) / prices.groupby(level=['symbol']).close  # target of X days
prices['5_day_target'] = prices.groupby(['symbol'])['close'].transform(lambda x:x.shift(-5))
prices['perc_change']=prices['5_day_target']/prices['close']
#prices['5dayvol'] = prices.groupby('symbol')['close']/prices.groupby(['symbol'])['close'].transform(lambda x:x.shift(1)).apply(pd.rolling_mean, 5, min_periods=5)  # 5 day volatility 

clean_prices = prices.loc[(prices['EMA_200'].notnull() & prices['5_day_target'].notnull())]


In [116]:
#prices.reset_index(level='date',inplace=True)

In [242]:
clean_prices.head()

open  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-10-16  19.31   
                                                          2000-10-17  19.00   
                                                          2000-10-18  17.88   
                                                          2000-10-19  17.94   
                                                          2000-10-20  17.75   

                                                                       high  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-10-16  19.31   
                                                          2000-10-17  19.00   
                                                          2000-10-18  17.88   
                                                          2000-10-19  17.97   
                                                          2000-10-20  17.75   

                                                                        low  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-10-16  19.06   
                                                          2000-10-17  17.88   
                                                          2000-10-18  17.56   
                                                          2000-10-19  17.38   
                                                          2000-10-20  17.50   

                                                                        close  \
symbol name                                               date                  
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-10-16  10.8956   
                                                          2000-10-17  10.1518   
                                                          2000-10-18   9.9701   
                                                          2000-10-19   9.8679   
                                                          2000-10-20  10.0780   

                                                                       volume  \
symbol name                                               date                  
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-10-16   4900.0   
                                                          2000-10-17  26700.0   
                                                          2000-10-18  13300.0   
                                                          2000-10-19  31500.0   
                                                          2000-10-20  14500.0   

                                                                         EMA_20  \
symbol name                                               date                    
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-10-16  11.126843   
                                                          2000-10-17  11.033982   
                                                          2000-10-18  10.932660   
                                                          2000-10-19  10.831254   
                                                          2000-10-20  10.759515   

                                                                         EMA_50  \
symbol name                                               date                    
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-10-16  10.769057   
                                                          2000-10-17  10.744851   
                                                          2000-10-18  10.714469   
                                                          2000-10-19  10.681270   
                                                          2000-10-20  10.657612   

                                                                        EMA_200  \
symbol name               

In [211]:
test.head(500)

open  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  25.50   
                                                          2000-01-04  25.00   
                                                          2000-01-05  24.75   
                                                          2000-01-06  24.00   
                                                          2000-01-07  23.88   
                                                          2000-01-10  24.00   
                                                          2000-01-11  23.88   
                                                          2000-01-12  24.13   
                                                          2000-01-13  23.63   
                                                          2000-01-14  23.50   
                                                          2000-01-18  23.19   
                                                          2000-01-19  22.88   
                                                          2000-01-20  23.00   
                                                          2000-01-21  22.38   
                                                          2000-01-24  22.25   
                                                          2000-01-25  21.78   
                                                          2000-01-26  21.81   
                                                          2000-01-27  21.63   
                                                          2000-01-28  20.63   
                                                          2000-01-31  20.50   
                                                          2000-02-01  20.00   
                                                          2000-02-02  19.94   
                                                          2000-02-03  19.75   
                                                          2000-02-04  19.00   
                                                          2000-02-07  20.63   
                                                          2000-02-08  20.25   
                                                          2000-02-09  20.00   
                                                          2000-02-10  20.25   
                                                          2000-02-11  20.25   
                                                          2000-02-14  19.88   
...                                                                     ...   
                                                          2001-11-16  21.51   
                                                          2001-11-19  21.53   
                                                          2001-11-20  21.39   
                                                          2001-11-21  21.52   
                                                          2001-11-23  21.65   
                                                          2001-11-26  21.63   
                                                          2001-11-27  21.17   
                                                          2001-11-28  21.46   
                                                          2001-11-29  21.09   
                                                          2001-11-30  21.24   
                                                          2001-12-03  20.70   
                                                          2001-12-04  20.70   
                                                          2001-12-05  20.73   
                                                          2001-12-06  21.37   
                                                          2001-12-07  21.49   
                                                          2001-12-10  21.10   
                                                          2001-12-11  20.50   
                                                          2001-12-12  20.71   
                                          

In [213]:
list(clean_prices.columns.values)

['open',
 'high',
 'low',
 'close',
 'volume',
 'EMA_20',
 'EMA_50',
 'EMA_200',
 'EMA_ratio',
 'RSI',
 'CCI',
 'Momentum',
 'Stoch_fastk',
 'Stoch_fastd',
 'OBV',
 'bbandupper',
 'bbandmiddle',
 'bbandlower',
 'bbandlowerii',
 'bbandupperii',
 '3 MO',
 '6 MO',
 '1 YR',
 '2 YR',
 '3 YR',
 '5 YR',
 '7 YR',
 '10 YR',
 '30 YR',
 '5_day_target',
 'perc_change']

In [214]:
testML=clean_prices.loc[:,[
 #'open',
 #'high',
 #'low',
 #'close',
 #'volume',
 #'EMA_20',
 #'EMA_50',
 #'EMA_200',
 'EMA_ratio',
 'RSI',
 'CCI',
 'Momentum',
 'Stoch_fastk',
 'Stoch_fastd',
 'OBV',
 #'bbandupper',
 #'bbandmiddle',
 #'bbandlower',
 'bbandlowerii',
 'bbandupperii',
 '3 MO',
 '6 MO',
 '1 YR',
 '2 YR',
 '3 YR',
 '5 YR',
 '7 YR',
 '10 YR',
 #'30 YR',
'perc_change']]

In [170]:
testML[testML.isnull().any(axis=1)].head()

In [216]:
testML=testML.fillna(method='pad')

In [217]:
    #x_train, x_test, y_train, y_test = train_test_split(clean_prices.loc[:,'EMA_ratio':'bbandlower'], clean_prices.loc[:,['7_day_target']], shuffle=False, test_size=90)
    test_size = 3
    # Get columns
    X = testML.copy().loc[:,'EMA_ratio':'10 YR'] 
    y = testML.copy().loc[:, ['perc_change']]

In [218]:
    # Get series of ingroup rownumbers
    X['num']= X.groupby(level=1).cumcount(ascending=False)
    
    testML.loc[ testML.EMA_ratio == testML.EMA_ratio.max() ] 

,,,EMA_ratio,RSI,CCI,Momentum,Stoch_fastk,Stoch_fastd,OBV,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,perc_change
symbol,name,date,,,,,,,,,,,,,,,,,,
ALLY,"Ally Financial Inc. (ALLY) Stock Prices, Dividends and Splits",2015-01-09,15.947293,59.149339,104.141459,0.0096,27.328947,24.068768,59407170.0,0.954267,0.954267,0.02,0.08,0.22,0.59,0.96,1.45,1.75,1.98,0.854272


In [219]:
    # Filter
    X_train = X.loc[X.num > test_size ].drop(columns='num')
    X_test =  X.loc[X.num <= test_size ].drop(columns='num')
    y_train = y.loc[X.num > test_size ]
    y_test =  y.loc[X.num <= test_size ]
    
    X_test.head(15)

EMA_ratio  \
symbol name                                               date                    
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2018-05-09   0.961522   
                                                          2018-05-10   0.961180   
                                                          2018-05-11   0.960818   
                                                          2018-05-14   0.960848   
XXII   22nd Century Group Inc. (XXII) Stock Prices, Di... 2018-05-09   1.006703   
                                                          2018-05-10   1.006708   
                                                          2018-05-11   1.007208   
                                                          2018-05-14   1.008681   
JOBS   51job Inc. (JOBS) Stock Prices, Dividends and S... 2018-05-09   0.811219   
                                                          2018-05-10   0.809703   
                                                          2018-05-11   0.808451   
                                                          2018-05-14   0.807096   
AIR    AAR Corp. (AIR) Stock Prices, Dividends and Splits 2018-05-09   0.924918   
                                                          2018-05-10   0.923591   
                                                          2018-05-11   0.922554   

                                                                            RSI  \
symbol name                                               date                    
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2018-05-09  58.334741   
                                                          2018-05-10  56.998784   
                                                          2018-05-11  57.651565   
                                                          2018-05-14  51.960195   
XXII   22nd Century Group Inc. (XXII) Stock Prices, Di... 2018-05-09  54.855877   
                                                          2018-05-10  54.372809   
                                                          2018-05-11  52.385610   
                                                          2018-05-14  48.562839   
JOBS   51job Inc. (JOBS) Stock Prices, Dividends and S... 2018-05-09  75.085824   
                                                          2018-05-10  66.025451   
                                                          2018-05-11  65.192559   
                                                          2018-05-14  66.543116   
AIR    AAR Corp. (AIR) Stock Prices, Dividends and Splits 2018-05-09  70.288377   
                                                          2018-05-10  70.331640   
                                                          2018-05-11  67.071209   

                                                                             CCI  \
symbol name                                               date                     
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2018-05-09  111.122117   
                                                          2018-05-10   66.067211   
                                                          2018-05-11   72.172128   
                                                          2018-05-14 -108.619173   
XXII   22nd Century Group Inc. (XXII) Stock Prices, Di... 2018-05-09   67.375887   
                                                          2018-05-10    5.865103   
                                                          2018-05-11  -56.801196   
                                                          2018-05-14 -102.040816   
JOBS   51job Inc. (JOBS) Stock Prices, Dividends and S... 2018-05-09   93.563843   
                                                          2018-05-10   62.830727   
                                                          2018-05-11  -18.761602   
                                                          2018-05-14   17.339167   
AIR    AAR Corp. (AIR) Stock Prices, Dividends and Splits 2018-05-09  

In [220]:
    %timeit
    forest = RandomForestRegressor()
    forest.fit(X_train,y_train)
    y_pred = forest.predict(X_test)
    y_pred = pd.DataFrame({'ans': y_pred})
    

C:\Users\Student\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [221]:
test.head()

open  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  25.50   
                                                          2000-01-04  25.00   
                                                          2000-01-05  24.75   
                                                          2000-01-06  24.00   
                                                          2000-01-07  23.88   

                                                                       high  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  25.50   
                                                          2000-01-04  25.00   
                                                          2000-01-05  24.81   
                                                          2000-01-06  24.00   
                                                          2000-01-07  23.88   

                                                                        low  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  24.75   
                                                          2000-01-04  24.75   
                                                          2000-01-05  23.75   
                                                          2000-01-06  23.63   
                                                          2000-01-07  23.50   

                                                                        close  \
symbol name                                               date                  
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  14.0034   
                                                          2000-01-04  13.8634   
                                                          2000-01-05  13.4433   
                                                          2000-01-06  13.3032   
                                                          2000-01-07  13.2360   

                                                                       volume  \
symbol name                                               date                  
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  17800.0   
                                                          2000-01-04   6300.0   
                                                          2000-01-05  20400.0   
                                                          2000-01-06   4200.0   
                                                          2000-01-07   9200.0   

                                                                      EMA_20  \
symbol name                                               date                 
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03     NaN   
                                                          2000-01-04     NaN   
                                                          2000-01-05     NaN   
                                                          2000-01-06     NaN   
                                                          2000-01-07     NaN   

                                                                      EMA_50  \
symbol name                                               date                 
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03     NaN   
                                                          2000-01-04     NaN   
                                                          2000-01-05     NaN   
                                                          2000-01-06     NaN   
                                                          2000-01-07     NaN   

                                                                      EMA_200  \
symbol name                                               date        

In [224]:
#final = pd.DataFrame({'close':test.tail(90).reset_index().close, 'ratio':y_pred['ans']})
#final['pred_price']= final['close'] * final['ratio']

In [228]:
final.head()

,close,ratio,pred_price
0,62.1667,0.990655,61.585758
1,61.7128,1.009679,62.310113
2,61.6635,1.021093,62.964193
3,63.0250,1.013045,63.847155
4,63.0941,0.983723,62.067105


In [243]:
ml=test.sort_index(level=1, ascending=True)
ml=test.groupby(['name']).tail(1).loc[:,'EMA_ratio':'10 YR']  #tail to get the last date entry for each day

In [245]:
ml.head()

,,,EMA_ratio,RSI,CCI,Momentum,Stoch_fastk,Stoch_fastd,OBV,bbandupper,bbandmiddle,bbandlower,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR
symbol,name,date,,,,,,,,,,,,,,,,,,,,
SRCE,"1st Source Corporation (SRCE) Stock Prices, Dividends and Splits",2018-05-21,0.958910,59.784249,76.797386,1.480,92.871312,89.100278,6280191.0,54.132806,53.4080,52.683194,0.978878,0.978878,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
XXII,"22nd Century Group Inc. (XXII) Stock Prices, Dividends and Splits",2018-05-21,1.018590,42.746257,-87.005176,-0.201,19.833333,22.722222,59994451.0,2.347342,2.1938,2.040258,0.962840,0.962840,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
JOBS,"51job Inc. (JOBS) Stock Prices, Dividends and Splits",2018-05-21,0.797592,69.411155,45.215962,7.160,89.578714,89.692143,24780106.0,106.101432,103.5100,100.918568,0.968044,0.968044,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
AIR,"AAR Corp. (AIR) Stock Prices, Dividends and Splits",2018-05-21,0.917865,71.608834,160.676533,2.580,99.508358,89.799562,12222538.0,47.947921,46.9100,45.872079,0.957663,0.957663,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
ABAX,"ABAXIS Inc. (ABAX) Stock Prices, Dividends and Splits",2018-05-21,0.859011,74.859553,41.555721,14.365,95.425014,97.175571,17931528.0,90.363771,81.0530,71.742229,0.862961,0.862961,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06


In [246]:
ml[ml.isnull().any(axis=1)].head()

,,,EMA_ratio,RSI,CCI,Momentum,Stoch_fastk,Stoch_fastd,OBV,bbandupper,bbandmiddle,bbandlower,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR
symbol,name,date,,,,,,,,,,,,,,,,,,,,
HCT,"American Realty Capital Healthcare Trust Inc (HCT) Stock Prices, Dividends and Splits",2015-01-16,NaN,74.897745,91.339156,1.1194,93.014192,92.029938,90514573.0,13.140772,13.027,12.913228,0.984615,0.984615,0.03,0.07,0.17,0.49,0.80,1.29,1.60,1.83
IMED,"PureFunds ETFx HealthTech ETF (IMED) Stock Prices, Dividends and Splits",2017-09-06,NaN,NaN,NaN,NaN,NaN,NaN,225.0,NaN,NaN,NaN,NaN,NaN,1.07,1.17,1.24,1.30,1.42,1.69,1.93,2.10
TIA,"Telecom Italia S.P.A. New (TI.A) Stock Prices, Dividends and Splits",2018-05-21,NaN,NaN,NaN,NaN,NaN,NaN,68864.0,NaN,NaN,NaN,NaN,NaN,1.93,2.14,2.35,2.58,2.73,2.90,3.02,3.06


In [251]:
ml=ml.loc[ml['EMA_ratio'].notnull()]

In [254]:
ml.drop(['bbandupper','bbandmiddle','bbandlower'], axis=1, inplace=True)

In [255]:
ml['5dayforecast'] = forest.predict(ml)

In [259]:
ml.head()

,,,EMA_ratio,RSI,CCI,Momentum,Stoch_fastk,Stoch_fastd,OBV,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,5dayforecast
symbol,name,date,,,,,,,,,,,,,,,,,,
SRCE,"1st Source Corporation (SRCE) Stock Prices, Dividends and Splits",2018-05-21,0.958910,59.784249,76.797386,1.480,92.871312,89.100278,6280191.0,0.978878,0.978878,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,1.038010
XXII,"22nd Century Group Inc. (XXII) Stock Prices, Dividends and Splits",2018-05-21,1.018590,42.746257,-87.005176,-0.201,19.833333,22.722222,59994451.0,0.962840,0.962840,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,1.003516
JOBS,"51job Inc. (JOBS) Stock Prices, Dividends and Splits",2018-05-21,0.797592,69.411155,45.215962,7.160,89.578714,89.692143,24780106.0,0.968044,0.968044,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,1.013578
AIR,"AAR Corp. (AIR) Stock Prices, Dividends and Splits",2018-05-21,0.917865,71.608834,160.676533,2.580,99.508358,89.799562,12222538.0,0.957663,0.957663,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,0.978834
ABAX,"ABAXIS Inc. (ABAX) Stock Prices, Dividends and Splits",2018-05-21,0.859011,74.859553,41.555721,14.365,95.425014,97.175571,17931528.0,0.862961,0.862961,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,1.023412


In [260]:
prices.head()

open  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  25.50   
                                                          2000-01-04  25.00   
                                                          2000-01-05  24.75   
                                                          2000-01-06  24.00   
                                                          2000-01-07  23.88   

                                                                       high  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  25.50   
                                                          2000-01-04  25.00   
                                                          2000-01-05  24.81   
                                                          2000-01-06  24.00   
                                                          2000-01-07  23.88   

                                                                        low  \
symbol name                                               date                
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  24.75   
                                                          2000-01-04  24.75   
                                                          2000-01-05  23.75   
                                                          2000-01-06  23.63   
                                                          2000-01-07  23.50   

                                                                        close  \
symbol name                                               date                  
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  14.0034   
                                                          2000-01-04  13.8634   
                                                          2000-01-05  13.4433   
                                                          2000-01-06  13.3032   
                                                          2000-01-07  13.2360   

                                                                       volume  \
symbol name                                               date                  
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03  17800.0   
                                                          2000-01-04   6300.0   
                                                          2000-01-05  20400.0   
                                                          2000-01-06   4200.0   
                                                          2000-01-07   9200.0   

                                                                      EMA_20  \
symbol name                                               date                 
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03     NaN   
                                                          2000-01-04     NaN   
                                                          2000-01-05     NaN   
                                                          2000-01-06     NaN   
                                                          2000-01-07     NaN   

                                                                      EMA_50  \
symbol name                                               date                 
SRCE   1st Source Corporation (SRCE) Stock Prices, Div... 2000-01-03     NaN   
                                                          2000-01-04     NaN   
                                                          2000-01-05     NaN   
                                                          2000-01-06     NaN   
                                                          2000-01-07     NaN   

                                                                      EMA_200  \
symbol name                                               date        

In [272]:
result=ml.join(prices, lsuffix='l',how ='left')

In [274]:
result[['close','5dayforecast']]

,,,close,5dayforecast
symbol,name,date,,
SRCE,"1st Source Corporation (SRCE) Stock Prices, Dividends and Splits",2018-05-21,53.8200,1.038010
XXII,"22nd Century Group Inc. (XXII) Stock Prices, Dividends and Splits",2018-05-21,2.1190,1.003516
JOBS,"51job Inc. (JOBS) Stock Prices, Dividends and Splits",2018-05-21,104.2500,1.013578
AIR,"AAR Corp. (AIR) Stock Prices, Dividends and Splits",2018-05-21,47.9000,0.978834
ABAX,"ABAXIS Inc. (ABAX) Stock Prices, Dividends and Splits",2018-05-21,83.1350,1.023412
ABB,"ABB Ltd (ABB) Stock Prices, Dividends and Splits",2018-05-21,24.4900,1.027369
ADTN,"ADTRAN Inc. (ADTN) Stock Prices, Dividends and Splits",2018-05-21,NaN,1.004067
AEG,"AEGON N.V. (AEG) Stock Prices, Dividends and Splits",2018-05-21,6.9017,0.991804
MITT,"AG Mortgage Investment Trust Inc. (MITT) Stock Prices, Dividends and Splits",2018-05-21,18.7000,1.009378


In [276]:
result=result[result.close.notnull()]

In [279]:
result=result[['close','5dayforecast']]

In [280]:
result.head()

,,,close,5dayforecast
symbol,name,date,,
SRCE,"1st Source Corporation (SRCE) Stock Prices, Dividends and Splits",2018-05-21,53.820,1.038010
XXII,"22nd Century Group Inc. (XXII) Stock Prices, Dividends and Splits",2018-05-21,2.119,1.003516
JOBS,"51job Inc. (JOBS) Stock Prices, Dividends and Splits",2018-05-21,104.250,1.013578
AIR,"AAR Corp. (AIR) Stock Prices, Dividends and Splits",2018-05-21,47.900,0.978834
ABAX,"ABAXIS Inc. (ABAX) Stock Prices, Dividends and Splits",2018-05-21,83.135,1.023412


In [281]:
result['5day_price']=result['close']*result['5dayforecast']

In [283]:
result['5day%_change']=(result['5dayforecast']-1)*100

In [284]:
result

,,,close,5dayforecast,5day_price,5day%_change
symbol,name,date,,,,
SRCE,"1st Source Corporation (SRCE) Stock Prices, Dividends and Splits",2018-05-21,53.8200,1.038010,55.865694,3.800993
XXII,"22nd Century Group Inc. (XXII) Stock Prices, Dividends and Splits",2018-05-21,2.1190,1.003516,2.126451,0.351640
JOBS,"51job Inc. (JOBS) Stock Prices, Dividends and Splits",2018-05-21,104.2500,1.013578,105.665500,1.357794
AIR,"AAR Corp. (AIR) Stock Prices, Dividends and Splits",2018-05-21,47.9000,0.978834,46.886128,-2.116642
ABAX,"ABAXIS Inc. (ABAX) Stock Prices, Dividends and Splits",2018-05-21,83.1350,1.023412,85.081333,2.341172
ABB,"ABB Ltd (ABB) Stock Prices, Dividends and Splits",2018-05-21,24.4900,1.027369,25.160256,2.736857
AEG,"AEGON N.V. (AEG) Stock Prices, Dividends and Splits",2018-05-21,6.9017,0.991804,6.845133,-0.819603
MITT,"AG Mortgage Investment Trust Inc. (MITT) Stock Prices, Dividends and Splits",2018-05-21,18.7000,1.009378,18.875374,0.937830
AGNC,"AGNC Investment Corp. (AGNC) Stock Prices, Dividends and Splits",2018-05-21,19.0000,1.037496,19.712415,3.749550


In [292]:
prep = result[['close','5day_price','5day%_change']]
prep.sort_values("5day%_change",ascending=False, inplace=True)
prep.reset_index(inplace=True)
prep.set_index('name')
tdf=timedelta(days=5)
prep=prep[prep['date']>=(datetime.now()-tdf)]

C:\Users\Student\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [293]:
prep.tail(10)

,symbol,name,date,close,5day_price,5day%_change
173,STM,"STMicroelectronics N.V. (STM) Stock Prices, Di...",2018-05-21,23.8600,23.520549,-1.422678
175,AAV,"Advantage Oil & Gas Ltd (AAV) Stock Prices, Di...",2018-05-21,3.1500,3.099196,-1.612818
176,AOI,Alliance One International Inc. (AOI) Stock Pr...,2018-05-21,18.7500,18.366048,-2.047744
177,AIR,"AAR Corp. (AIR) Stock Prices, Dividends and Sp...",2018-05-21,47.9000,46.886128,-2.116642
178,SSW,"Seaspan Corporation (SSW) Stock Prices, Divide...",2018-05-21,8.8900,8.662733,-2.556436
179,AEO,American Eagle Outfitters Inc. (AEO) Stock Pri...,2018-05-21,22.9600,22.284187,-2.943436
180,ACST,"Acasti Pharma Inc. (ACST) Stock Prices, Divide...",2018-05-21,0.7498,0.723994,-3.441770
181,TCF,"TCF Financial Corporation (TCF) Stock Prices, ...",2018-05-21,26.5900,25.494426,-4.120250
182,SSI,"Stage Stores Inc. (SSI) Stock Prices, Dividend...",2018-05-21,3.1500,3.006181,-4.565677
183,ATV,Acorn International Inc. ADS (ATV) Stock Price...,2018-05-21,26.6959,24.813562,-7.051036


In [294]:
prep.head(10)

,symbol,name,date,close,5day_price,5day%_change
0,AMDA,"Amedica Corporation (AMDA) Stock Prices, Divid...",2018-05-21,1.1400,1.409156,23.610204
2,AXE,"Anixter International Inc. (AXE) Stock Prices,...",2018-05-21,63.6000,69.809108,9.762748
3,ADS,Alliance Data Systems Corporation (ADS) Stock ...,2018-05-21,213.8300,231.083931,8.068995
4,SNA,"Snap-On Incorporated (SNA) Stock Prices, Divid...",2018-05-21,154.8600,167.027436,7.857055
5,ALB,"Albemarle Corporation (ALB) Stock Prices, Divi...",2018-05-21,103.8448,111.621113,7.488399
6,UHAL,"Amerco (UHAL) Stock Prices, Dividends and Splits",2018-05-21,355.2800,377.405730,6.227688
7,AUTO,"AutoWeb Inc. (AUTO) Stock Prices, Dividends an...",2018-05-21,3.6600,3.885626,6.164657
8,TGT,"Target Corporation (TGT) Stock Prices, Dividen...",2018-05-21,77.0600,81.806581,6.159592
9,ANTH,Anthera Pharmaceuticals Inc. (ANTH) Stock Pric...,2018-05-21,0.2879,0.303381,5.377371
10,TREX,"Trex Company Inc. (TREX) Stock Prices, Dividen...",2018-05-21,118.1500,124.088254,5.026029
